In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import haversine as hs
import geopandas as gp
from shapely.geometry import Point, Polygon
import numpy as np

from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from pyomo.environ import Block, Expression, units as pyunits
import math
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
from truck_pipe_cost_functions import truck_costing, pipe_costing
from bw_lcow_functions import *
from shapely import wkt

In [2]:
# Import files
# Brackish water
state_alpha = 'AL'
bw_df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_baseline_dwi_5Feb23.csv')
PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\clustering_results\\' + state_alpha
cluster_bw_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_cluster.csv').drop(['Unnamed: 0'],axis=1)
bw_cluster_kmeans = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_bw_kmeans.csv').drop(['Unnamed: 0'],axis=1)

In [3]:
# Analysis for a single state
# Read public water demand data at each place
PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha
pubsupply_demand_df = pd.read_csv(PATH + '\\' + state_alpha.lower() + '_pubsupply_demand_df.csv', index_col = 0)
pubsupply_elev_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_pubsupply_elevation_data.csv')

In [2]:
# pubsupply_demand_df.head(2)

In [44]:
# Function to calculate bw and public supply table without clustering
# Selects wells in increasing order of LCOW
def state_ps_calc(state_alpha):

    # Get state sub-dataframe
    bw_state_df = bw_df[bw_df['state_alpha']== state_alpha]
    bw_state_df = bw_state_df.set_index('unique_site_ID')

    # Read state elevation dataset
    bw_state_elev = pd.read_csv(r'\Users\mhardika\Documents\AMO\GeoToolAll_Methods\Water Source Data\Brackish\elevation_data\\'+ state_alpha.lower() + '.csv',index_col='unique_site_ID').drop(['Unnamed: 0'],axis=1)

    # Read demand data set
    PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha
    pubsupply_demand_df = pd.read_csv(PATH + '\\' + state_alpha.lower() + '_pubsupply_demand_df.csv', index_col = 0)
    pubsupply_demand_df = pubsupply_demand_df.set_index('place_code')
    # Read the centroids column as points
    # centroids = pubsupply_demand_df['centroid'].apply(wkt.loads)

    # Read elevation dataset
    try:
        pubsupply_elev_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_pubsupply_elevation_data_updated.csv')
        print('Using updated')
    except:
        pubsupply_elev_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_pubsupply_elevation_data.csv')
    pubsupply_elev_df = pubsupply_elev_df.set_index('place_code')

    cols = ['State','PlaceCode','PSLatitude','PSLongitude','PSDemand','BWSites','BWLatitude','BWLongitude','BWAvailable','Distance_km',
        'PSDemandFracMet','LCOW','TransportLCOW','tLCOW','WeightedtLCOW','Cum_PSDemandFracMet']

    one_ps_many_bw_df  = pd.DataFrame(columns=cols)

    for place in pubsupply_demand_df.index.drop_duplicates():
        # print(place)
        ps_long = pubsupply_demand_df['centroid'].drop_duplicates().apply(wkt.loads).loc[place].x #centroids.loc[place].x
        ps_lat = pubsupply_demand_df['centroid'].drop_duplicates().apply(wkt.loads).loc[place].y #centroids.loc[place].y
        try:
            ps_elev = pubsupply_elev_df.loc[place]['elevation_m'].values[0]
        except AttributeError:
            ps_elev = pubsupply_elev_df.loc[place]['elevation_m']

        ps_loc = (ps_lat,ps_long)   

        for site in bw_state_df.index:
            # print(site)
            bw_long = bw_state_df['Longitude'].loc[site]
            bw_lat = bw_state_df['Latitude'].loc[site]
            bw_elev = bw_state_elev.loc[site]['elevation_m']
            # try:
            #     bw_elev = bw_cluster_kmeans[bw_cluster_kmeans['unique_site_ID']==site]['well_elevation'].values[0]
            # except:
            #     bw_elev = bw_cluster_kmeans[bw_cluster_kmeans['unique_site_ID']==site]['well_elevation'].values
            #     print(bw_elev)
            bw_loc = (bw_lat,bw_long)
            dist_km = hs.haversine(ps_loc,bw_loc)

            bw_available = bw_state_df['well_yield'].loc[site]*bw_state_df['recovery'].loc[site]/100
            ps_demand = pubsupply_demand_df.loc[place]['place_water_demand (m3/yr)'].sum()/(365.25*24*3600)  # m3/s
            ps_demand_frac_met = bw_available/(ps_demand)

            # This includes treatment and brine disposal
            lcow = bw_state_df['lcow'].loc[site] + bw_state_df['well_field_lcow'].loc[site]/(bw_state_df['recovery'].loc[site]/100)

            # Updated transport costs
            elev_gain = ps_elev - bw_elev

            if elev_gain<0:
                elev_gain = 1e-5 

            # Cost of transport to point of use. Both are m3/s
            if bw_available>ps_demand:
                transport_lcow_pipe = pipe_costing(ps_demand*3600*24,dist_km,elev_gain= elev_gain,
                                               electricity_rate=bw_state_df['elec_price'].loc[site])
                transportvol = ps_demand
            else:
                transport_lcow_pipe = pipe_costing(bw_available*3600*24,dist_km,elev_gain= elev_gain,
                                               electricity_rate=bw_state_df['elec_price'].loc[site])
                transportvol = bw_available
            
            tlcow = lcow + transport_lcow_pipe

            temp_dict ={'State' : state_alpha,
                        'PlaceCode' : place,
                        'PSLatitude' : ps_lat,
                        'PSLongitude' : ps_long,
                        'PSElevation' : ps_elev,
                        'PSDemand' : ps_demand,
                        'BWSites' : site,
                        'BWLatitude' : bw_lat,
                        'BWLongitude' : bw_long,
                        'BWAvailable' : bw_available,
                        'BWElevation' : bw_elev,
                        'Distance_km' : dist_km,
                        'PSDemandFracMet' : ps_demand_frac_met,
                        'LCOW' : lcow,
                        'TransportVol_m3/s': transportvol,
                        'TransportLCOW' : transport_lcow_pipe,
                        'tLCOW' : tlcow,
                        'WeightedtLCOW': 0,
                        'Cum_PSDemandFracMet': 0}
            
            temp = pd.DataFrame(temp_dict,index=[0])    
            one_ps_many_bw_df = pd.concat([one_ps_many_bw_df,temp],ignore_index=True)
            one_ps_many_bw_df.reset_index()

    weighted_lcow = []
    cum_ps_demand_frac_met = []

    one_ps_many_bw_df_sorted1 = one_ps_many_bw_df.sort_values(['PlaceCode','tLCOW']).groupby('PlaceCode')
    one_ps_many_bw_df = one_ps_many_bw_df_sorted1.apply(pd.DataFrame).set_index('PlaceCode')


    for place in pubsupply_demand_df.index.drop_duplicates():
        cum_bw_available = np.array(one_ps_many_bw_df['BWAvailable'].loc[place].cumsum())
        cum_cost = np.array((one_ps_many_bw_df['BWAvailable'].loc[place]*one_ps_many_bw_df['tLCOW'].loc[place]).cumsum())
        temp = np.divide(cum_cost,cum_bw_available)
        weighted_lcow = np.append(weighted_lcow, temp,axis=0)
        temp_demand_met = np.array(one_ps_many_bw_df['PSDemandFracMet'].loc[place].cumsum())
        cum_ps_demand_frac_met = np.append(cum_ps_demand_frac_met,temp_demand_met,axis=0)


    one_ps_many_bw_df['WeightedtLCOW'] = weighted_lcow
    one_ps_many_bw_df['Cum_PSDemandFracMet'] = cum_ps_demand_frac_met
    one_ps_many_bw_df['Cum_PSDemandFracMet'] = np.where(one_ps_many_bw_df['Cum_PSDemandFracMet']>1,1,one_ps_many_bw_df['Cum_PSDemandFracMet'])      

    return (one_ps_many_bw_df)

In [27]:
# PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha
# pubsupply_demand_df = pd.read_csv(PATH + '\\' + state_alpha.lower() + '_pubsupply_demand_df.csv', index_col = 0)
# pubsupply_demand_df = pubsupply_demand_df.set_index('place_code')
# # centroids = pubsupply_demand_df['centroid'].apply(wkt.loads)
# # pubsupply_demand_df['centroid'].drop_duplicates().apply(wkt.loads).loc[3455].x
# # pubsupply_demand_df.loc[4000]['place_water_demand (m3/yr)'].sum()

# pubsupply_elev_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_pubsupply_elevation_data.csv', index_col = 0)
# pubsupply_elev_df = pubsupply_elev_df.set_index('place_code')

# pubsupply_elev_df.loc[44275]['elevation_m']
# pubsupply_demand_df.index.drop_duplicates()

In [50]:
# Running unclustered scenarios
# state_codes = ['AL','AZ','AR','CA','CO','CT','FL','GA','ID','IL',
#                'IN','IA','KS','KY','LA','MD','MI','MN','MS','MO',
#                'MA','MT','NE','NV','NJ','NM','NY','NC','ND','OH',
#                'NH','OK','OR','PA','SC','SD','TX','UT','VA','WA',
#                'TN','WV','WI','WY','ME']


# for state_alpha in ['AZ']: #state_codes[31:45]:
#     print(state_alpha)
#     one_ps_many_bw_df = state_ps_calc(state_alpha)

#     PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha
#     if not os.path.exists(PATH):
#         os.makedirs(PATH)

#     name = '\ps_unclustered_' + state_alpha.lower()+'.csv'

#     one_ps_many_bw_df.to_csv(PATH + '\\' + name )

In [31]:
pipe_costing(0.00039*3600*24,47,elev_gain= 20,
                                               electricity_rate=0.05)

91.49131260857045

In [13]:
# Function to calculate bw and public supply table for clustering
# Selects wells in increasing order of LCOW

def state_ps_cluster_calc(state_alpha, check = False):
    
    # Get state sub-dataframe
    PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\clustering_results\\' + state_alpha
    cluster_bw_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_cluster.csv').drop(['Unnamed: 0'],axis=1)
    bw_state_df = cluster_bw_df.set_index('cluster_id')

    # Read demand data set
    PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha
    pubsupply_demand_df = pd.read_csv(PATH + '\\' + state_alpha.lower() + '_pubsupply_demand_df.csv', index_col = 0)
    pubsupply_demand_df = pubsupply_demand_df.set_index('place_code')

    # Read elevation dataset
    try:
        pubsupply_elev_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_pubsupply_elevation_data_updated.csv')
        print('Using updated')
    except:
        pubsupply_elev_df = pd.read_csv(PATH + '\\' + state_alpha.lower()+'_pubsupply_elevation_data.csv')

    pubsupply_elev_df = pubsupply_elev_df.set_index('place_code')

    cols = ['State','PlaceCode','PSLatitude','PSLongitude','PSDemand','BWSites','BWLatitude','BWLongitude','BWAvailable','Distance_km',
        'PSDemandFracMet','LCOW','TransportVol_m3/s','TransportLCOW','tLCOW','WeightedtLCOW','Cum_PSDemandFracMet']

    one_ps_many_bw_df  = pd.DataFrame(columns=cols)

    for place in pubsupply_demand_df.index.drop_duplicates():
        if check == True:
            print(place)
        ps_long = pubsupply_demand_df['centroid'].drop_duplicates().apply(wkt.loads).loc[place].x #centroids.loc[place].x
        ps_lat = pubsupply_demand_df['centroid'].drop_duplicates().apply(wkt.loads).loc[place].y #centroids.loc[place].y
        try:
            ps_elev = pubsupply_elev_df.loc[place]['elevation_m'].values[0]
        except AttributeError:
            ps_elev = pubsupply_elev_df.loc[place]['elevation_m']

        ps_loc = (ps_lat,ps_long)   

        for site in bw_state_df.index:
            bw_long = bw_state_df['centroid_long'].loc[site]
            bw_lat = bw_state_df['centroid_lat'].loc[site]
            bw_elev = bw_state_df['centroid_elev'].loc[site]
            bw_loc = (bw_lat,bw_long)
            dist_km = hs.haversine(ps_loc,bw_loc)

            bw_available = bw_state_df['cluster_total_well_yield'].loc[site]*bw_state_df['recovery'].loc[site]/100
            ps_demand = pubsupply_demand_df.loc[place]['place_water_demand (m3/yr)'].sum()/(365.25*24*3600)  # m3/s
            ps_demand_frac_met = bw_available/(ps_demand)

            # This includes treatment and brine disposal
            lcow = bw_state_df['cluster_lcow'].loc[site]
            
            # Updated transport costs
            elev_gain = ps_elev - bw_elev
            try:
                if elev_gain < 0.0:
                    elev_gain = 1e-5 
            except:
                print(ps_elev)
            # Cost of transport to point of use. Both are m3/s
            if bw_available>ps_demand:
                transport_lcow_pipe = pipe_costing(ps_demand*3600*24,dist_km,elev_gain= elev_gain,
                                               electricity_rate=bw_state_df['elec_price'].loc[site])
                transportvol = ps_demand
            else:
                transport_lcow_pipe = pipe_costing(bw_available*3600*24,dist_km,elev_gain= elev_gain,
                                               electricity_rate=bw_state_df['elec_price'].loc[site])
                transportvol = bw_available
            
            # transport_lcow_pipe_check = pipe_costing(bw_available*3600*24,dist_km,elev_gain= elev_gain,
            #                                    electricity_rate=bw_state_df['elec_price'].loc[site])
            
            tlcow = lcow + transport_lcow_pipe

            temp_dict ={'State' : state_alpha,
                        'PlaceCode' : place,
                        'PSLatitude' : ps_lat,
                        'PSLongitude' : ps_long,
                        'PSElevation' : ps_elev,
                        'PSDemand' : ps_demand,
                        'BWSites' : site,
                        'BWLatitude' : bw_lat,
                        'BWLongitude' : bw_long,
                        'BWAvailable' : bw_available,
                        'BWElevation' : bw_elev,
                        'Distance_km' : dist_km,
                        'PSDemandFracMet' : ps_demand_frac_met,
                        'LCOW' : lcow,
                        'TransportVol_m3/s': transportvol,
                        'TransportLCOW' : transport_lcow_pipe,
                        'tLCOW' : tlcow,
                        'WeightedtLCOW': 0,
                        'Cum_PSDemandFracMet': 0}
            
            # print(temp_dict)
            temp = pd.DataFrame(temp_dict,index=[0])    
            one_ps_many_bw_df = pd.concat([one_ps_many_bw_df,temp],ignore_index=True)
            one_ps_many_bw_df.reset_index()

    weighted_lcow = []
    cum_ps_demand_frac_met = []

    one_ps_many_bw_df_sorted1 = one_ps_many_bw_df.sort_values(['PlaceCode','tLCOW']).groupby('PlaceCode')
    one_ps_many_bw_df = one_ps_many_bw_df_sorted1.apply(pd.DataFrame).set_index('PlaceCode')


    for place in pubsupply_demand_df.index.drop_duplicates():
        cum_bw_available = np.array(one_ps_many_bw_df['BWAvailable'].loc[place].cumsum())
        cum_cost = np.array((one_ps_many_bw_df['BWAvailable'].loc[place]*one_ps_many_bw_df['tLCOW'].loc[place]).cumsum())
        temp = np.divide(cum_cost,cum_bw_available)
        weighted_lcow = np.append(weighted_lcow, temp,axis=0)
        temp_demand_met = np.array(one_ps_many_bw_df['PSDemandFracMet'].loc[place].cumsum())
        cum_ps_demand_frac_met = np.append(cum_ps_demand_frac_met,temp_demand_met,axis=0)

    one_ps_many_bw_df['WeightedtLCOW'] = weighted_lcow
    one_ps_many_bw_df['Cum_PSDemandFracMet'] = cum_ps_demand_frac_met
    one_ps_many_bw_df['Cum_PSDemandFracMet'] = np.where(one_ps_many_bw_df['Cum_PSDemandFracMet']>1,1,one_ps_many_bw_df['Cum_PSDemandFracMet'])      

    return (one_ps_many_bw_df)

In [78]:
# one_ps_many_bw_cluster_df = state_ps_cluster_calc('FL',check=True)

375
1775
25175
29275
30525
37300
45225
48200
74925
26075
41950
9725
11150
37500
41562
41825
44300
54725
54700
55075
58962
68275
72875
73312
8725
28575
39700
68525
10250
13175
13225
13150
27256
33375
33450
35800
42625
44000
44075
43975
44275
45275
45775
49435
54000
54425
55375
58700
61500
64400
64625
65525
67725
71900
72521
74368
74370
76058
76500
7687
8650
13275
14125
14400
16335
16475
16725
24000
24710
28452
30850
30858
32000
39525
39475
39550
39750
40450
43125
45975
49425
50575
55125
55750
55775
57425
58050
61807
64725
68135
69700
70675
75225
76658
76582
78000
1000
6925
11625
11650
12950
20825
27775
28800
42750
58350
59200
61937
66875
6125
6667
12412
12450
15775
22775
29425
32375
32400
33958
33966
33950
39825
57058
68950
2187
5025
22660
27400
36475
38813
45350
50630
52125
55875
2677
5806
12150
21425
26300
26775
29381
33250
34185
39987
40037
43083
43090
47650
47675
47625
52162
55650
55680
57066
57492
74235
74562
78063
22475
24500
37775
75300
1750
67355
15575
32650
2400
3250
35050
3500

In [61]:
# in v2, the transport cost is calculated using the minimum (bw_available and ps_demand)

In [28]:
state_codes = ['AL','AZ','AR','CA','CO','CT','FL','GA','ID','IL',
               #'IN','MA','NH','TN','ME',
               'IA','KS','KY','LA','MD','MI','MN','MS','MO',
               'MT','NE','NV','NJ','NM','NY','NC','ND','OH',
               'OK','OR','PA','SC','SD','TX','UT','VA','WA',
               'WV','WI','WY']


for state_alpha in state_codes[37:41]:
    print(state_alpha)
    one_ps_many_bw_cluster_df = state_ps_cluster_calc(state_alpha)

    PATH = r'\Users\mhardika\Documents\AMO\Analysis Data Files\Public Supply\state_wise_analysis\\' + state_alpha
    if not os.path.exists(PATH):
        os.makedirs(PATH)

    name = '\ps_clustered_v2_' + state_alpha.lower()+'.csv'

    one_ps_many_bw_cluster_df.to_csv(PATH + '\\' + name)


WV
WI
Using updated
WY
ME


FileNotFoundError: [Errno 2] No such file or directory: '\\Users\\mhardika\\Documents\\AMO\\Analysis Data Files\\clustering_results\\\\ME\\me_cluster.csv'